In [31]:
import numpy as np
import h3

h = h3.geo_to_h3(0,0,5)

In [32]:
h

'85754e67fffffff'

In [56]:
def get_siblings(h):
    p = h3.h3_to_parent(h)
    sibs = h3.h3_to_children(p)
    sibs = sorted(sibs)
    
    return sibs

def get_next_sibling(h):
    """ Returns None if h is the last child
    """
    sibs = get_siblings(h)
    i = sibs.index(h) + 1
    
    if i >= len(sibs):
        return None
    else:
        return sibs[i]
    
def get_prev_sibling(h):
    """ Returns None if h is the first child
    """
    sibs = get_siblings(h)
    i = sibs.index(h) - 1
    
    if i < 0:
        return None
    else:
        return sibs[i]
    
def is_a_first_child(h):
    if h3.h3_get_resolution(h) == 0:
        return False

    if get_prev_sibling(h) is None:
        return True
    
def is_first_descendent_of(p, h):
    res_p = h3.h3_get_resolution(p)
    res_h = h3.h3_get_resolution(h)
    
    if res_p <= res_h and h3.h3_to_parent(h, res_p):
        return is_a_first_child(h)
    else:
        return False
    
def siblings_up_to(h):
    sibs = get_siblings(h)
    for sib in sibs:
        yield sib
        if sib == h:
            break

In [65]:
get_siblings(h)

['85754e63fffffff',
 '85754e67fffffff',
 '85754e6bfffffff',
 '85754e6ffffffff',
 '85754e73fffffff',
 '85754e77fffffff',
 '85754e7bfffffff']

In [171]:
# b = list(h3.get_res0_indexes())[0]
# hexes = h3.h3_to_children(b, 1)

h = h3.geo_to_h3(0,0,5)

inputs = get_siblings(h)#[h]
stack = []
out = []

def step():
    global stack
    global out
    global inputs
    
    
    if stack:
        h = stack[-1]
        # completed children set
        if get_next_sibling(h) is None:  # maybe need to worry about res 0 cells...
            stack.pop()
            inputs = [h3.h3_to_parent(h)] + inputs
            return # end so that we can run step again
        
    
    h = inputs[0]
    inputs = inputs[1:]

    if not stack:
        if is_a_first_child(h):
            stack.append(h)
            return
        else:
            out.append(h)
            return
    
    if stack:
        next_hex = get_next_sibling(stack[-1])
        
        if h == next_hex:
            stack[-1] = h # what if it completes the set?
            return
        elif is_first_descendent_of(next_hex, h):
            stack.append(h)
        else:
            # flush the stack
            for e in stack:
                for h in siblings_up_to(e):
                    out.append(h)
            stack[:] = []
    

In [187]:
inputs

['85754e6ffffffff', '85754e73fffffff', '85754e77fffffff', '85754e7bfffffff']

In [212]:
step()

IndexError: list index out of range

In [213]:
inputs

[]

In [214]:
stack

[]

In [215]:
out

['84754e7ffffffff']